In [9]:
import pandas as pd
import json
import ast
from pathlib import Path
import os

### Read data from individual csv files

In [10]:
print(os.getcwd())

/home/paul_d/Sources/sem_math_repo/classification


In [11]:
# positive examples for classification
#  binary dataset: algebra-precalculus
#  pos_sample_path = Path(".") / "data" / "formula_data_algebra-precalculus.csv" 

pos_sample_path = Path("..") / "data_intermediate1" / "formula_data_elementary-set-theory.csv"
elem_set_th_df = pd.read_csv(pos_sample_path)
elem_set_th_df.head()

,fid,mtype,exprstr,mention,tokens,type_tokens,tags
0,6327106106a69a3f488d6e3e_f_8_2_0,FUNC,f,any function,['f'],"['func_name', '__ANON_1']","['elementary-set-theory', 'intuition', 'infini..."
1,6327106106a69a3f488d6e96_f_3_22_0,FUNC,"j: \mathbb{N} \rightarrow [0, 1]",any injection f_3_22_0,"['j', ':', '\\mathbb{N}', '\\rightarrow', '[',...","['func_def', 'func_name', '__ANON_1', 'COLON',...",['elementary-set-theory']
2,6327106106a69a3f488d6e96_f_3_38_0,FUNC,b,together the identity map,['b'],"['func_name', '__ANON_1']",['elementary-set-theory']
3,6327106106a69a3f488d6ef4_f_0_0_0,SET,\mathbb{Q},the set f_0_0_0,\mathbb{Q},"['set_constant', 'SET_BASIC']","['elementary-set-theory', 'irrational-numbers']"
4,6327106106a69a3f488d6f48_f_0_0_0,FUNC,f,a given function,['f'],"['func_name', '__ANON_1']","['functions', 'elementary-set-theory', 'functi..."


In [12]:
# negative examples for classification 

# binary dataset: algebra-precalculus
# elem_set_th_df = pd.read_csv("data/formula_data_elementary-set-theory.csv")

# binary dataset: elemenary-set-theory
algebra_precalc_df = Path("..") / "data_intermediate1" / "formula_data_algebra-precalculus.csv"

a_geometry_df = Path("..") / "data_intermediate1" / "formula_data_analytic-geometry.csv"
elem_func_df = Path("..") / "data_intermediate1" / "formula_data_elementary-functions.csv"
elem_num_th_df = Path("..") / "data_intermediate1" / "formula_data_elementary-number-theory.csv" 
euc_geom_df = Path("..") / "data_intermediate1" / "formula_data_euclidean-geometry.csv" 
trig_df = Path("..") / "data_intermediate1" / "formula_data_trigonometry.csv"

algebra_precalc_df = pd.read_csv(algebra_precalc_df)
a_geometry_df = pd.read_csv(a_geometry_df)
elem_func_df = pd.read_csv(elem_func_df)
elem_num_th_df = pd.read_csv(elem_num_th_df)
euc_geom_df = pd.read_csv(euc_geom_df)
trig_df = pd.read_csv(trig_df)

neg_df_list = [algebra_precalc_df, a_geometry_df, elem_func_df, elem_num_th_df, euc_geom_df, trig_df]
lengths_list = [d_frame.shape[0] for d_frame in neg_df_list]
print(lengths_list)

[5917, 234, 182, 6253, 330, 1705]


#### Merge negative examples datasets while keeping the occurence frequencies and filter

In [13]:
def merge_dfs(df_list, return_size):
    r_nums = [d_frame.shape[0] for d_frame in df_list]
    r_nums_sum = sum(r_nums)

    sel_neg_dfs = [df_list[i].sample(n=int((r_nums[i]/r_nums_sum)*return_size)) for i in range(len(r_nums))]
    neg_samples = pd.concat(sel_neg_dfs, axis=0)
    if neg_samples.shape[0] < return_size:
        neg_samples = pd.concat([neg_samples,df_list[-1].sample(n=return_size-neg_samples.shape[0])])
    return neg_samples

In [14]:
def filter_data(train_df, is_pos):
    def cell_to_str(cell_val):
        if isinstance(cell_val,int):
            return "['" + str(cell_val) + "']"
        elif isinstance(cell_val,float):
            return "['" + str(cell_val) + "']"
        elif isinstance(cell_val,str):
            if len(cell_val) == 1:
                return "['" + cell_val + "']"
            else:
                return cell_val
        else:
            return cell_val

    train_df = train_df.drop(["fid", "exprstr", "mention", "tags"], axis=1)
    train_df["tokens"] = train_df["tokens"].map(cell_to_str)

    if is_pos:
        train_df["label"] = 1
    else:
        train_df["label"] = 0
    return train_df

In [15]:
intended_train_size = 11000 
neg_samples_df = merge_dfs(neg_df_list, int(intended_train_size/2))
neg_samples_df.head()
print("negative examples size: ", neg_samples_df.shape[0])
pos_samples_df = elem_set_th_df.sample(n=int(intended_train_size/2))
print("positive examples size: ", pos_samples_df.shape[0])


negative examples size:  5500
positive examples size:  5500


In [16]:
pos_samples_df = filter_data(pos_samples_df, True)
neg_samples_df = filter_data(neg_samples_df, False)

data_df = pd.concat([pos_samples_df, neg_samples_df], axis=0)
print("total size: ", data_df.shape[0])
data_df.head()

total size:  11000


,mtype,tokens,type_tokens,label
1117,FUNC,"['f', ':', 'A', '\\to', 'B']","['func_def', 'func_name', '__ANON_1', 'COLON',...",1
2408,FUNC,['f'],"['func_name', '__ANON_1']",1
3965,FUNC,"['g', '\\colon', 'B', '\\to', 'I']","['func_def', 'func_name', '__ANON_1', 'COLON',...",1
5954,SCAL,"['\\frac', '{', '8', '!', '}', '{', '(', '8', ...","['expr_atom', 'CMD_FRAC', 'L_BRACE', 'postfix'...",1
3127,FUNC,['f'],"['func_name', '__ANON_1']",1


In [17]:
output_path = Path("..") / "data" / "bin_class_data_TOKENIZED_SET_V1.csv"
data_df.to_csv(output_path, index=False)

#### Remove single - token formulas from dataset

In [30]:
if isinstance(data_df.iloc[0,1], str):
    print("hey")
elif isinstance(data_df.iloc[0,1], list):
    print("list")
else:
    print("not hey")
    print(data_df.iloc[0,1])

hey


In [33]:
some_df = data_df.loc[data_df["tokens"] == "['f']"]
some_df.head()

,mtype,tokens,type_tokens,label
2408,FUNC,['f'],"['func_name', '__ANON_1']",1
3127,FUNC,['f'],"['func_name', '__ANON_1']",1
8476,FUNC,['f'],"['func_name', '__ANON_1']",1
8860,FUNC,['f'],"['func_name', '__ANON_1']",1
6714,FUNC,['f'],"['func_name', '__ANON_1']",1


In [36]:

clean_data_df = data_df.loc[data_df["tokens"].str.len() > 5]
clean_data_df.head()

,mtype,tokens,type_tokens,label
1117,FUNC,"['f', ':', 'A', '\\to', 'B']","['func_def', 'func_name', '__ANON_1', 'COLON',...",1
3965,FUNC,"['g', '\\colon', 'B', '\\to', 'I']","['func_def', 'func_name', '__ANON_1', 'COLON',...",1
5954,SCAL,"['\\frac', '{', '8', '!', '}', '{', '(', '8', ...","['expr_atom', 'CMD_FRAC', 'L_BRACE', 'postfix'...",1
4674,FUNC,"['g', ':', 'A', '\\rightarrow', 'B']","['func_def', 'func_name', '__ANON_1', 'COLON',...",1
8485,FUNC,"['f', ':', 'X', '\\to', 'Y']","['func_def', 'func_name', '__ANON_1', 'COLON',...",1


In [37]:
output_path = Path("..") / "data" / "bin_class_data_TOKENIZED_SET_V1_compact.csv"
clean_data_df.to_csv(output_path, index=False)